In [1]:
import keras
import sys
sys.path.append('/home/reiichiro/ipython_notebooks/Project/files for cv')
import numpy as np
import joblib
from sklearn.grid_search import GridSearchCV
from sklearn.linear_model import LogisticRegression
from myscorer import my_custom_scorer_neural
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.callbacks import ProgbarLogger
from keras.callbacks import EarlyStopping

np.random.seed(0)

Using Theano backend.
/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
complete_features_matrix = joblib.load('Pickles/Condensed_Basic+Temporal+Recency/train.pkl')
complete_features_matrix

matrix([[ 0.8909911 ,  0.75005068,  0.7497068 , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.8909911 ,  0.75005068,  0.7497068 , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.8909911 ,  0.75005068,  0.7497068 , ...,  0.        ,
          0.        ,  0.        ],
        ..., 
        [ 0.88798993,  0.88168444,  0.88168444, ...,  0.        ,
          0.        ,  0.        ],
        [ 0.88798993,  0.88168444,  0.88168444, ...,  0.        ,
          0.        ,  0.        ],
        [ 0.88798993,  0.88168444,  0.88168444, ...,  0.        ,
          0.        ,  0.        ]])

In [3]:
labels = joblib.load('Pickles/labels.pkl')

In [4]:
train_indices, test_indices = joblib.load('Pickles/cv_indices')
cv_iterable = [(train_indices, test_indices)]

In [5]:
def build_fn(no_hidden=14, hidden_activation='sigmoid', out_activation='sigmoid', 
             loss='mse', optimizer='rmsprop', random_state=0, 
             callbacks=[EarlyStopping(monitor='val_loss', patience=2)], 
             validation_split=0.2):
    
    np.random.seed(0)

    model = Sequential([
        Dense(no_hidden, input_dim=28),
        Activation(hidden_activation),
        Dense(1),
        Activation(out_activation),
    ])
    

    model.compile(loss=loss, optimizer=optimizer)
    return model

In [6]:
clf = KerasClassifier(build_fn)
clf

In [7]:
param_grid = [
  {'batch_size': [128], 'nb_epoch': [1, 2, 3], 
   'hidden_activation': ['relu', 'sigmoid'], 'optimizer': ['rmsprop', 'sgd']}
 ]

In [8]:
grid_search = GridSearchCV(clf, param_grid, cv=cv_iterable, scoring=my_custom_scorer_neural, n_jobs=1, verbose=10, refit=False)

In [9]:
grid_search.fit(complete_features_matrix, labels)

Fitting 1 folds for each of 12 candidates, totalling 12 fits
[CV] hidden_activation=relu, optimizer=rmsprop, nb_epoch=1, batch_size=128 
Epoch 1/1
482432/487006 [============================>.] - ETA: 0s[CV]  hidden_activation=relu, optimizer=rmsprop, nb_epoch=1, batch_size=128, score=-0.296573 - 2.2min
[CV] hidden_activation=relu, optimizer=sgd, nb_epoch=1, batch_size=128 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  2.2min remaining:    0.0s


Epoch 1/1
481536/487006 [============================>.] - ETA: 0s[CV]  hidden_activation=relu, optimizer=sgd, nb_epoch=1, batch_size=128, score=-0.310359 - 3.6min
[CV] hidden_activation=relu, optimizer=rmsprop, nb_epoch=2, batch_size=128 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  5.8min remaining:    0.0s


Epoch 1/2
7996914/7996914 [==============================] - 106s - loss: 0.0977   
Epoch 2/2
487006/487006 [==============================] - 0s     
[CV]  hidden_activation=relu, optimizer=rmsprop, nb_epoch=2, batch_size=128, score=-0.297961 - 4.8min
[CV] hidden_activation=relu, optimizer=sgd, nb_epoch=2, batch_size=128 


...

Epoch 1/2
7996914/7996914 [==============================] - 116s - loss: 0.0970   
Epoch 2/2
484992/487006 [============================>.] - ETA: 0s[CV]  hidden_activation=sigmoid, optimizer=rmsprop, nb_epoch=2, batch_size=128, score=-0.297519 - 4.6min
[CV] hidden_activation=sigmoid, optimizer=sgd, nb_epoch=2, batch_size=128 


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed: 39.4min remaining:    0.0s


Epoch 1/2
7996914/7996914 [==============================] - 95s - loss: 0.1121    
Epoch 2/2
487006/487006 [==============================] - 1s     
[CV]  hidden_activation=sigmoid, optimizer=sgd, nb_epoch=2, batch_size=128, score=-0.307883 - 6.3min
[CV] hidden_activation=sigmoid, optimizer=rmsprop, nb_epoch=3, batch_size=128 
Epoch 1/3
7996914/7996914 [==============================] - 111s - loss: 0.0970   
Epoch 2/3
7996914/7996914 [==============================] - 49s - loss: 0.0949    
Epoch 3/3
486144/487006 [============================>.] - ETA: 0s[CV]  hidden_activation=sigmoid, optimizer=rmsprop, nb_epoch=3, batch_size=128, score=-0.296831 - 7.4min
[CV] hidden_activation=sigmoid, optimizer=sgd, nb_epoch=3, batch_size=128 
Epoch 1/3
7996914/7996914 [==============================] - 140s - loss: 0.1121   
Epoch 2/3
7996914/7996914 [==============================] - 45s - loss: 0.1047    
Epoch 3/3
487006/487006 [==============================] - 0s     
[CV]  hidden_activat

[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed: 59.5min finished


GridSearchCV(cv=[(array([      0,       1, ..., 8483911, 8483919], dtype=int32), array([     31,      32, ..., 8483917, 8483918], dtype=int32))],
       error_score='raise',
       estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x7f784c479b50>,
       fit_params={}, iid=True, n_jobs=1,
       param_grid=[{'nb_epoch': [1, 2, 3], 'optimizer': ['rmsprop', 'sgd'], 'hidden_activation': ['relu', 'sigmoid'], 'batch_size': [128]}],
       pre_dispatch='2*n_jobs', refit=False,
       scoring=<function my_custom_scorer_neural at 0x7f78222809b0>,
       verbose=10)

In [10]:
grid_search.grid_scores_

[mean: -0.29657, std: 0.00000, params: {'hidden_activation': 'relu', 'optimizer': 'rmsprop', 'nb_epoch': 1, 'batch_size': 128},
 mean: -0.31036, std: 0.00000, params: {'hidden_activation': 'relu', 'optimizer': 'sgd', 'nb_epoch': 1, 'batch_size': 128},
 mean: -0.29796, std: 0.00000, params: {'hidden_activation': 'relu', 'optimizer': 'rmsprop', 'nb_epoch': 2, 'batch_size': 128},
 mean: -0.30529, std: 0.00000, params: {'hidden_activation': 'relu', 'optimizer': 'sgd', 'nb_epoch': 2, 'batch_size': 128},
 mean: -0.29958, std: 0.00000, params: {'hidden_activation': 'relu', 'optimizer': 'rmsprop', 'nb_epoch': 3, 'batch_size': 128},
 mean: -0.30511, std: 0.00000, params: {'hidden_activation': 'relu', 'optimizer': 'sgd', 'nb_epoch': 3, 'batch_size': 128},
 mean: -0.29618, std: 0.00000, params: {'hidden_activation': 'sigmoid', 'optimizer': 'rmsprop', 'nb_epoch': 1, 'batch_size': 128},
 mean: -0.31250, std: 0.00000, params: {'hidden_activation': 'sigmoid', 'optimizer': 'sgd', 'nb_epoch': 1, 'batch

In [11]:
joblib.dump(grid_search, 'Pickles/Condensed_Basic+Temporal+Recency/GridSearches/NNs/1.pkl', compress=3)

['Pickles/Condensed_Basic+Temporal+Recency/GridSearches/NNs/1.pkl']

### more grids, same xcept for adam

In [13]:
param_grid = [
  {'batch_size': [128], 'nb_epoch': [1, 2, 3], 
   'hidden_activation': ['relu', 'sigmoid'], 'optimizer': ['adam']}
 ]

In [14]:
grid_search = GridSearchCV(clf, param_grid, cv=cv_iterable, scoring=my_custom_scorer_neural, n_jobs=1, verbose=10, refit=False)

In [15]:
grid_search.fit(complete_features_matrix, labels)

Fitting 1 folds for each of 6 candidates, totalling 6 fits
[CV] hidden_activation=relu, optimizer=adam, nb_epoch=1, batch_size=128 
Epoch 1/1
482432/487006 [============================>.] - ETA: 0s[CV]  hidden_activation=relu, optimizer=adam, nb_epoch=1, batch_size=128, score=-0.299532 - 4.3min
[CV] hidden_activation=relu, optimizer=adam, nb_epoch=2, batch_size=128 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  4.3min remaining:    0.0s


Epoch 1/2
7996914/7996914 [==============================] - 143s - loss: 0.0974   
Epoch 2/2
480000/487006 [============================>.] - ETA: 0s[CV]  hidden_activation=relu, optimizer=adam, nb_epoch=2, batch_size=128, score=-0.294983 - 5.9min
[CV] hidden_activation=relu, optimizer=adam, nb_epoch=3, batch_size=128 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed: 10.2min remaining:    0.0s


Epoch 1/3
7996914/7996914 [==============================] - 156s - loss: 0.0974   
Epoch 2/3
7996914/7996914 [==============================] - 58s - loss: 0.0948    
Epoch 3/3
482944/487006 [============================>.] - ETA: 0s[CV]  hidden_activation=relu, optimizer=adam, nb_epoch=3, batch_size=128, score=-0.294372 - 6.7min
[CV] hidden_activation=sigmoid, optimizer=adam, nb_epoch=1, batch_size=128 


...

Epoch 1/2
7996914/7996914 [==============================] - 132s - loss: 0.0972   
Epoch 2/2
483840/487006 [============================>.] - ETA: 0s[CV]  hidden_activation=sigmoid, optimizer=adam, nb_epoch=2, batch_size=128, score=-0.294601 - 8.9min
[CV] hidden_activation=sigmoid, optimizer=adam, nb_epoch=3, batch_size=128 


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed: 33.0min remaining:    0.0s


Epoch 1/3
7996914/7996914 [==============================] - 112s - loss: 0.0972   
Epoch 2/3
7996914/7996914 [==============================] - 53s - loss: 0.0947    
Epoch 3/3
482560/487006 [============================>.] - ETA: 0s[CV]  hidden_activation=sigmoid, optimizer=adam, nb_epoch=3, batch_size=128, score=-0.294223 - 8.9min


[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed: 41.8min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed: 41.8min finished


GridSearchCV(cv=[(array([      0,       1, ..., 8483911, 8483919], dtype=int32), array([     31,      32, ..., 8483917, 8483918], dtype=int32))],
       error_score='raise',
       estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x7f784c479b50>,
       fit_params={}, iid=True, n_jobs=1,
       param_grid=[{'nb_epoch': [1, 2, 3], 'optimizer': ['adam'], 'hidden_activation': ['relu', 'sigmoid'], 'batch_size': [128]}],
       pre_dispatch='2*n_jobs', refit=False,
       scoring=<function my_custom_scorer_neural at 0x7f78222809b0>,
       verbose=10)

In [16]:
grid_search.grid_scores_

[mean: -0.29953, std: 0.00000, params: {'hidden_activation': 'relu', 'optimizer': 'adam', 'nb_epoch': 1, 'batch_size': 128},
 mean: -0.29498, std: 0.00000, params: {'hidden_activation': 'relu', 'optimizer': 'adam', 'nb_epoch': 2, 'batch_size': 128},
 mean: -0.29437, std: 0.00000, params: {'hidden_activation': 'relu', 'optimizer': 'adam', 'nb_epoch': 3, 'batch_size': 128},
 mean: -0.29750, std: 0.00000, params: {'hidden_activation': 'sigmoid', 'optimizer': 'adam', 'nb_epoch': 1, 'batch_size': 128},
 mean: -0.29460, std: 0.00000, params: {'hidden_activation': 'sigmoid', 'optimizer': 'adam', 'nb_epoch': 2, 'batch_size': 128},
 mean: -0.29422, std: 0.00000, params: {'hidden_activation': 'sigmoid', 'optimizer': 'adam', 'nb_epoch': 3, 'batch_size': 128}]

In [17]:
joblib.dump(grid_search, 'Pickles/Condensed_Basic+Temporal+Recency/GridSearches/NNs/2.pkl', compress=3)

['Pickles/Condensed_Basic+Temporal+Recency/GridSearches/NNs/2.pkl']

## grid search incorporating early stopping

In [7]:
param_grid = [
  {'batch_size': [128, 256], 'nb_epoch': [10], 
   'hidden_activation': ['relu', 'sigmoid'], 
   'optimizer': ['rmsprop', 'sgd', 'adam'], 'callbacks': [[EarlyStopping(monitor='val_loss', patience=2)]], 
   'validation_split': [0.2]}
 ]

In [8]:
grid_search = GridSearchCV(clf, param_grid, cv=cv_iterable, scoring=my_custom_scorer_neural, n_jobs=1, verbose=10, refit=False)

In [9]:
grid_search.fit(complete_features_matrix, labels)

Fitting 1 folds for each of 12 candidates, totalling 12 fits
[CV] optimizer=rmsprop, hidden_activation=relu, batch_size=128, callbacks=[<keras.callbacks.EarlyStopping object at 0x7f47266ebc50>], validation_split=0.2, nb_epoch=10 
Train on 6397531 samples, validate on 1599383 samples
Epoch 1/10
6397531/6397531 [==============================] - 65s - loss: 0.0985 - val_loss: 0.0945
Epoch 2/10
6397531/6397531 [==============================] - 41s - loss: 0.0957 - val_loss: 0.0935
Epoch 3/10
6397531/6397531 [==============================] - 42s - loss: 0.0952 - val_loss: 0.0934
Epoch 4/10
6397531/6397531 [==============================] - 47s - loss: 0.0949 - val_loss: 0.0933
Epoch 5/10
6397531/6397531 [==============================] - 53s - loss: 0.0947 - val_loss: 0.0933
Epoch 6/10
6397531/6397531 [==============================] - 53s - loss: 0.0944 - val_loss: 0.0928
Epoch 7/10
6397531/6397531 [==============================] - 40s - loss: 0.0943 - val_loss: 0.0927
Epoch 8/10
63975

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  9.9min remaining:    0.0s


Train on 6397531 samples, validate on 1599383 samples
Epoch 1/10
6397531/6397531 [==============================] - 100s - loss: 0.1130 - val_loss: 0.1023
Epoch 2/10
6397531/6397531 [==============================] - 34s - loss: 0.1051 - val_loss: 0.1100
Epoch 3/10
6397531/6397531 [==============================] - 34s - loss: 0.1035 - val_loss: 0.1029
Epoch 4/10
6397531/6397531 [==============================] - 34s - loss: 0.1025 - val_loss: 0.0985
Epoch 5/10
6397531/6397531 [==============================] - 38s - loss: 0.1015 - val_loss: 0.0991
Epoch 6/10
6397531/6397531 [==============================] - 38s - loss: 0.1010 - val_loss: 0.0985
Epoch 7/10
6397531/6397531 [==============================] - 37s - loss: 0.1004 - val_loss: 0.0968
Epoch 8/10
6397531/6397531 [==============================] - 38s - loss: 0.0999 - val_loss: 0.0988
Epoch 9/10
6397531/6397531 [==============================] - 34s - loss: 0.0991 - val_loss: 0.0958
Epoch 10/10
481536/487006 [==================

[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed: 21.8min remaining:    0.0s


MemoryError: 

### "Grid search" for my low memory laptop

In [2]:
param_grid = [
  {'batch_size': [128, 256], 'nb_epoch': [10], 
   'hidden_activation': ['relu', 'sigmoid'], 
   'optimizer': ['rmsprop', 'sgd', 'adam'], 'callbacks': [[EarlyStopping(monitor='val_loss', patience=2)]], 
   'validation_split': [0.2]}
 ]

In [3]:
X_train = joblib.load('Pickles/Condensed_Basic+Temporal+Recency/X_train.pkl')
X_train.shape

(7996914, 28)

In [4]:
X_val = joblib.load('Pickles/Condensed_Basic+Temporal+Recency/X_val.pkl')
X_val.shape

(487006, 28)

In [5]:
y_train = joblib.load('Pickles/labels_train.pkl')
y_train.shape

(7996914,)

In [6]:
y_val = joblib.load('Pickles/labels_val.pkl')
y_val.shape

(487006,)

In [7]:
def build_fn(no_hidden=14, hidden_activation='sigmoid', out_activation='sigmoid', 
             loss='mse', optimizer='rmsprop', random_state=0):
    
    np.random.seed(0)

    model = Sequential([
        Dense(no_hidden, input_dim=28),
        Activation(hidden_activation),
        Dense(1),
        Activation(out_activation),
    ])
    

    model.compile(loss=loss, optimizer=optimizer)
    return model

In [8]:
grid_scores = {}
for batch_size in param_grid[0]['batch_size']:
    for nb_epoch in param_grid[0]['nb_epoch']:
        for hidden_activation in param_grid[0]['hidden_activation']:
            for optimizer in param_grid[0]['optimizer']:
                key = " ".join([str(batch_size), str(nb_epoch), str(hidden_activation), str(optimizer)])
                print("\n\nStarting CV for " + key)
                clf = KerasClassifier(build_fn, batch_size=batch_size, 
                                      nb_epoch=nb_epoch, hidden_activation=hidden_activation, 
                                      optimizer=optimizer, 
                                      callbacks=[EarlyStopping(monitor='val_loss', patience=2)], 
                                      validation_split=0.2)
                clf.fit(X_train, y_train)
                rmse = my_custom_scorer_neural(clf, X_val, y_val)
                grid_scores[key] = rmse
                print rmse



Starting CV for 128 10 relu rmsprop
Train on 6397531 samples, validate on 1599383 samples
Epoch 1/10
6397531/6397531 [==============================] - 47s - loss: 0.0985 - val_loss: 0.0945
Epoch 2/10
6397531/6397531 [==============================] - 41s - loss: 0.0957 - val_loss: 0.0935
Epoch 3/10
6397531/6397531 [==============================] - 38s - loss: 0.0952 - val_loss: 0.0934
Epoch 4/10
6397531/6397531 [==============================] - 39s - loss: 0.0949 - val_loss: 0.0933
Epoch 5/10
6397531/6397531 [==============================] - 44s - loss: 0.0947 - val_loss: 0.0933
Epoch 6/10
6397531/6397531 [==============================] - 43s - loss: 0.0944 - val_loss: 0.0928
Epoch 7/10
6397531/6397531 [==============================] - 43s - loss: 0.0943 - val_loss: 0.0927
Epoch 8/10
6397531/6397531 [==============================] - 43s - loss: 0.0942 - val_loss: 0.0932
Epoch 9/10
6397531/6397531 [==============================] - 42s - loss: 0.0942 - val_loss: 0.0929
Epoch 10

In [9]:
grid_scores

{'128 10 relu adam': -0.29331954213373795,
 '128 10 relu rmsprop': -0.29309132643991853,
 '128 10 relu sgd': -0.29827397858100779,
 '128 10 sigmoid adam': -0.29266022084908949,
 '128 10 sigmoid rmsprop': -0.2930172731217085,
 '128 10 sigmoid sgd': -0.29948291614577865,
 '256 10 relu adam': -0.29467964965361709,
 '256 10 relu rmsprop': -0.29345745105973514,
 '256 10 relu sgd': -0.31017672465606788,
 '256 10 sigmoid adam': -0.29343743092671648,
 '256 10 sigmoid rmsprop': -0.29334858681252246,
 '256 10 sigmoid sgd': -0.30275239571168105}

In [10]:
joblib.dump(grid_scores, 'Pickles/Condensed_Basic+Temporal+Recency/GridSearches/NNs/3(dict).pkl', compress=3)

['Pickles/Condensed_Basic+Temporal+Recency/GridSearches/NNs/3(dict).pkl']

### Low memory grid search experimenting with no hidden nodes

In [11]:
param_grid = [
  {'batch_size': [128], 'nb_epoch': [10], 
   'hidden_activation': ['relu', 'sigmoid'], 
   'optimizer': ['rmsprop', 'sgd', 'adam'], 'callbacks': [[EarlyStopping(monitor='val_loss', patience=2)]], 
   'validation_split': [0.2], 'no_hidden': [7, 18]}
 ]

def build_fn(no_hidden=14, hidden_activation='sigmoid', out_activation='sigmoid', 
             loss='mse', optimizer='rmsprop', random_state=0):
    
    np.random.seed(0)

    model = Sequential([
        Dense(no_hidden, input_dim=28),
        Activation(hidden_activation),
        Dense(1),
        Activation(out_activation),
    ])
    

    model.compile(loss=loss, optimizer=optimizer)
    return model

In [13]:
grid_scores = {}
for no_hidden in param_grid[0]['no_hidden']:
    for batch_size in param_grid[0]['batch_size']:
        for nb_epoch in param_grid[0]['nb_epoch']:
            for hidden_activation in param_grid[0]['hidden_activation']:
                for optimizer in param_grid[0]['optimizer']:
                    key = " ".join([str(no_hidden), str(batch_size), str(nb_epoch), 
                                    str(hidden_activation), str(optimizer)])
                    print("\n\nStarting CV for " + key)
                    clf = KerasClassifier(build_fn, no_hidden=no_hidden, batch_size=batch_size, 
                                          nb_epoch=nb_epoch, hidden_activation=hidden_activation, 
                                          optimizer=optimizer, 
                                          callbacks=[EarlyStopping(monitor='val_loss', patience=2)], 
                                          validation_split=0.2)
                    clf.fit(X_train, y_train)
                    rmse = my_custom_scorer_neural(clf, X_val, y_val)
                    grid_scores[key] = rmse
                    print rmse



Starting CV for 7 128 10 relu rmsprop
Train on 6397531 samples, validate on 1599383 samples
Epoch 1/10
6397531/6397531 [==============================] - 34s - loss: 0.0991 - val_loss: 0.0945
Epoch 2/10
6397531/6397531 [==============================] - 36s - loss: 0.0958 - val_loss: 0.0946
Epoch 3/10
6397531/6397531 [==============================] - 36s - loss: 0.0955 - val_loss: 0.0941
Epoch 4/10
6397531/6397531 [==============================] - 34s - loss: 0.0954 - val_loss: 0.0938
Epoch 5/10
6397531/6397531 [==============================] - 34s - loss: 0.0952 - val_loss: 0.0936
Epoch 6/10
6397531/6397531 [==============================] - 33s - loss: 0.0951 - val_loss: 0.0954
Epoch 7/10
6397531/6397531 [==============================] - 34s - loss: 0.0951 - val_loss: 0.0940
Epoch 8/10
6397531/6397531 [==============================] - 34s - loss: 0.0950 - val_loss: 0.0936
Epoch 9/10
6397531/6397531 [==============================] - 33s - loss: 0.0950 - val_loss: 0.0938
Epoch 

In [14]:
grid_scores

{'18 128 10 relu adam': -0.29316161078437991,
 '18 128 10 relu rmsprop': -0.29328991448706099,
 '18 128 10 relu sgd': -0.29816086401735004,
 '18 128 10 sigmoid adam': -0.29342983111917242,
 '18 128 10 sigmoid rmsprop': -0.29798769850731432,
 '18 128 10 sigmoid sgd': -0.29969652466774416,
 '7 128 10 relu adam': -0.29686766438411266,
 '7 128 10 relu rmsprop': -0.29474118465005522,
 '7 128 10 relu sgd': -0.30065997104743691,
 '7 128 10 sigmoid adam': -0.29398623225040749,
 '7 128 10 sigmoid rmsprop': -0.29511464121481618,
 '7 128 10 sigmoid sgd': -0.30026327784054518}

In [15]:
joblib.dump(grid_scores, 'Pickles/Condensed_Basic+Temporal+Recency/GridSearches/NNs/4(dict).pkl', compress=3)

['Pickles/Condensed_Basic+Temporal+Recency/GridSearches/NNs/4(dict).pkl']

### Low memory grid search experimenting with 2 hidden layers

In [17]:
param_grid = [
  {'batch_size': [128, 256], 'nb_epoch': [20], 
   'hidden_activation': ['relu', 'sigmoid'], 
   'optimizer': ['rmsprop', 'sgd', 'adam'], 'callbacks': [[EarlyStopping(monitor='val_loss', patience=2)]], 
   'validation_split': [0.2]}
 ]

In [18]:
def build_fn(no_hidden_1=18, no_hidden_2=8, hidden_activation='sigmoid', out_activation='sigmoid', 
             loss='mse', optimizer='rmsprop', random_state=0):
    
    np.random.seed(0)

    model = Sequential([
        Dense(no_hidden_1, input_dim=28),
        Activation(hidden_activation),
        Dense(no_hidden_2), 
        Activation(hidden_activation),
        Dense(1),
        Activation(out_activation),
    ])
    

    model.compile(loss=loss, optimizer=optimizer)
    return model

In [19]:
grid_scores = {}
ctr = 0
for batch_size in param_grid[0]['batch_size']:
    for nb_epoch in param_grid[0]['nb_epoch']:
        for hidden_activation in param_grid[0]['hidden_activation']:
            for optimizer in param_grid[0]['optimizer']:
                ctr += 1
                key = " ".join([str(batch_size), str(nb_epoch), str(hidden_activation), str(optimizer)])
                print("\n\nStarting CV for " + key), 1
                clf = KerasClassifier(build_fn, batch_size=batch_size, 
                                      nb_epoch=nb_epoch, hidden_activation=hidden_activation, 
                                      optimizer=optimizer, 
                                      callbacks=[EarlyStopping(monitor='val_loss', patience=2)], 
                                      validation_split=0.2)
                clf.fit(X_train, y_train)
                rmse = my_custom_scorer_neural(clf, X_val, y_val)
                grid_scores[key] = rmse
                print rmse



Starting CV for 128 20 relu rmsprop 1
Train on 6397531 samples, validate on 1599383 samples
Epoch 1/20
6397531/6397531 [==============================] - 59s - loss: 0.0974 - val_loss: 0.0941
Epoch 2/20
6397531/6397531 [==============================] - 61s - loss: 0.0952 - val_loss: 0.0952
Epoch 3/20
6397531/6397531 [==============================] - 58s - loss: 0.0947 - val_loss: 0.0932
Epoch 4/20
6397531/6397531 [==============================] - 52s - loss: 0.0946 - val_loss: 0.0939
Epoch 5/20
6397531/6397531 [==============================] - 52s - loss: 0.0945 - val_loss: 0.0932
Epoch 6/20
6397531/6397531 [==============================] - 56s - loss: 0.0944 - val_loss: 0.0927
Epoch 7/20
6397531/6397531 [==============================] - 61s - loss: 0.0942 - val_loss: 0.0932
Epoch 8/20
6397531/6397531 [==============================] - 60s - loss: 0.0941 - val_loss: 0.0925
Epoch 9/20
6397531/6397531 [==============================] - 61s - loss: 0.0940 - val_loss: 0.0926
Epoch 

In [20]:
grid_scores

{'128 20 relu adam': -0.29215648586630444,
 '128 20 relu rmsprop': -0.2941130103354781,
 '128 20 relu sgd': -0.29963217038595874,
 '128 20 sigmoid adam': -0.29182912138253392,
 '128 20 sigmoid rmsprop': -0.29394199237117868,
 '128 20 sigmoid sgd': -0.29815160165516991,
 '256 20 relu adam': -0.29220824151446928,
 '256 20 relu rmsprop': -0.29316710112333955,
 '256 20 relu sgd': -0.29864323117034253,
 '256 20 sigmoid adam': -0.29486395490022232,
 '256 20 sigmoid rmsprop': -0.29316420463096438,
 '256 20 sigmoid sgd': -0.30075673268512754}

In [21]:
joblib.dump(grid_scores, 'Pickles/Condensed_Basic+Temporal+Recency/GridSearches/NNs/5(dict).pkl', compress=3)

['Pickles/Condensed_Basic+Temporal+Recency/GridSearches/NNs/5(dict).pkl']

In [22]:
clf = KerasClassifier(build_fn, batch_size=256, 
                      nb_epoch=20, hidden_activation='sigmoid', 
                      optimizer='adam', 
                      validation_split=0.2)
clf.fit(X_train, y_train)
rmse = my_custom_scorer_neural(clf, X_val, y_val)
rmse

Train on 6397531 samples, validate on 1599383 samples
Epoch 1/20
6397531/6397531 [==============================] - 54s - loss: 0.0987 - val_loss: 0.0947
Epoch 2/20
6397531/6397531 [==============================] - 52s - loss: 0.0950 - val_loss: 0.0934
Epoch 3/20
6397531/6397531 [==============================] - 52s - loss: 0.0947 - val_loss: 0.0944
Epoch 4/20
6397531/6397531 [==============================] - 50s - loss: 0.0945 - val_loss: 0.0941
Epoch 5/20
6397531/6397531 [==============================] - 47s - loss: 0.0944 - val_loss: 0.0934
Epoch 6/20
6397531/6397531 [==============================] - 48s - loss: 0.0942 - val_loss: 0.0926
Epoch 7/20
6397531/6397531 [==============================] - 51s - loss: 0.0941 - val_loss: 0.0932
Epoch 8/20
6397531/6397531 [==============================] - 51s - loss: 0.0939 - val_loss: 0.0924
Epoch 9/20
6397531/6397531 [==============================] - 53s - loss: 0.0938 - val_loss: 0.0923
Epoch 10/20
6397531/6397531 [=================

-0.29258125686035824

### "Grid search" for my low memory laptop

In [7]:
param_grid = [
  {'batch_size': [128, 256], 'nb_epoch': [20], 
   'hidden_activation': ['relu', 'sigmoid'], 
   'optimizer': ['adagrad', 'adadelta', 'adamax', 'nadam'], 'callbacks': [[EarlyStopping(monitor='val_loss', patience=2)]], 
   'validation_split': [0.2]}
 ]

In [8]:
def build_fn(no_hidden=14, hidden_activation='sigmoid', out_activation='sigmoid', 
             loss='mse', optimizer='rmsprop', random_state=0):
    
    np.random.seed(0)

    model = Sequential([
        Dense(no_hidden, input_dim=28),
        Activation(hidden_activation),
        Dense(1),
        Activation(out_activation),
    ])
    

    model.compile(loss=loss, optimizer=optimizer)
    return model

In [9]:
grid_scores = {}
for batch_size in param_grid[0]['batch_size']:
    for nb_epoch in param_grid[0]['nb_epoch']:
        for hidden_activation in param_grid[0]['hidden_activation']:
            for optimizer in param_grid[0]['optimizer']:
                key = " ".join([str(batch_size), str(nb_epoch), str(hidden_activation), str(optimizer)])
                print("\n\nStarting CV for " + key)
                clf = KerasClassifier(build_fn, batch_size=batch_size, 
                                      nb_epoch=nb_epoch, hidden_activation=hidden_activation, 
                                      optimizer=optimizer, 
                                      callbacks=[EarlyStopping(monitor='val_loss', patience=2)], 
                                      validation_split=0.2, verbose=2)
                clf.fit(X_train, y_train)
                rmse = my_custom_scorer_neural(clf, X_val, y_val)
                grid_scores[key] = rmse
                print rmse



Starting CV for 128 20 relu adagrad
Train on 6397531 samples, validate on 1599383 samples
Epoch 1/20
30s - loss: 0.1008 - val_loss: 0.0970
Epoch 2/20
30s - loss: 0.0978 - val_loss: 0.0960
Epoch 3/20
29s - loss: 0.0970 - val_loss: 0.0953
Epoch 4/20
28s - loss: 0.0963 - val_loss: 0.0948
Epoch 5/20
28s - loss: 0.0960 - val_loss: 0.0947
Epoch 6/20
29s - loss: 0.0957 - val_loss: 0.0944
Epoch 7/20
28s - loss: 0.0956 - val_loss: 0.0943
Epoch 8/20
28s - loss: 0.0955 - val_loss: 0.0943
Epoch 9/20
28s - loss: 0.0954 - val_loss: 0.0942
Epoch 10/20
28s - loss: 0.0953 - val_loss: 0.0941
Epoch 11/20
28s - loss: 0.0952 - val_loss: 0.0940
Epoch 12/20
28s - loss: 0.0952 - val_loss: 0.0940
Epoch 13/20
29s - loss: 0.0951 - val_loss: 0.0939
Epoch 14/20
28s - loss: 0.0951 - val_loss: 0.0940
Epoch 15/20
28s - loss: 0.0950 - val_loss: 0.0938
Epoch 16/20
28s - loss: 0.0950 - val_loss: 0.0938
Epoch 17/20
28s - loss: 0.0950 - val_loss: 0.0938
Epoch 18/20
28s - loss: 0.0949 - val_loss: 0.0938
Epoch 19/20
28s -

In [10]:
grid_scores

{'128 20 relu adadelta': -0.29385568604943346,
 '128 20 relu adagrad': -0.29532124753027389,
 '128 20 relu adamax': -0.29455240879803946,
 '128 20 relu nadam': -0.29413420346303876,
 '128 20 sigmoid adadelta': -0.29343301227231011,
 '128 20 sigmoid adagrad': -0.29651971405943789,
 '128 20 sigmoid adamax': -0.29261940864883129,
 '128 20 sigmoid nadam': -0.2932575636638457,
 '256 20 relu adadelta': -0.29414278337656485,
 '256 20 relu adagrad': -0.29534116043652392,
 '256 20 relu adamax': -0.29572626411069064,
 '256 20 relu nadam': -0.29277247137921925,
 '256 20 sigmoid adadelta': -0.29368145191823447,
 '256 20 sigmoid adagrad': -0.29662033571804203,
 '256 20 sigmoid adamax': -0.29446303085456554,
 '256 20 sigmoid nadam': -0.29297769809882479}

In [11]:
joblib.dump(grid_scores, 'Pickles/Condensed_Basic+Temporal+Recency/GridSearches/NNs/6(dict).pkl', compress=3)

['Pickles/Condensed_Basic+Temporal+Recency/GridSearches/NNs/6(dict).pkl']